In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets
from torch.utils import data

In [3]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [4]:
EPOCHS = 30
BATCH_SIZE = 64

In [5]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])
test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

In [6]:
trainset = datasets.FashionMNIST(
    root = './.data/',
    train = True,
    transform = train_transform,
    download=True
)

100.0%

Extracting ./.data/FashionMNIST\raw\train-images-idx3-ubyte.gz to ./.data/FashionMNIST\raw


111.0%

Extracting ./.data/FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./.data/FashionMNIST\raw


100.0%

Extracting ./.data/FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./.data/FashionMNIST\raw


159.1%..\torch\csrc\utils\tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


Extracting ./.data/FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data/FashionMNIST\raw
Processing...
Done!


In [7]:
testset = datasets.FashionMNIST(
    root = './.data/',
    train = False,
    transform = test_transform,
    download=True
)

In [8]:
train_loader = data.DataLoader(
    dataset = trainset,
    batch_size=BATCH_SIZE)
test_loader = data.DataLoader(
    dataset = testset,
    batch_size= BATCH_SIZE
)

In [9]:
class Net(nn.Module):
    def __init__(self, dropout_p=0.2):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128,10)
        self.dropout_p = dropout_p
        
    def forward(self, input):
        flatten = input.view(-1, 784)
        dropped1 = F.dropout(flatten, training=self.training, p=self.dropout_p )
        fc1 = F.relu(self.fc1(dropped1))
        dropped2 = F.dropout(fc1, training=self.training, p=self.dropout_p)
        fc2 = F.relu(self.fc2(dropped2))
        output = self.fc3(fc2)
        return output

In [10]:
model = Net(dropout_p=0.2).to(DEVICE)

In [11]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [12]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
    
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()

In [13]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)
    
    return test_loss, test_accuracy

In [14]:
test   = torch.Tensor([[0,0,1,0,0,0,0,0,0],[0,0,0,0,1,0,0,0,0],[1,0,0,0,0,0,0,0,0]])

In [15]:
test.max(1)

torch.return_types.max(
values=tensor([1., 1., 1.]),
indices=tensor([2, 4, 0]))

In [16]:
test.max(1, keepdim=True)[1].shape

torch.Size([3, 1])

In [17]:
for epoch in range(1, EPOCHS+1):
    train(model, train_loader, optimizer)
    test_loss, test_accuracy = evaluate(model, test_loader)
    
    print(f'{epoch:2d} Test Loss : {test_loss:.4f}, accuracy : {test_accuracy:.2f}')

 1 Test Loss : 0.6344, accuracy : 77.70
 2 Test Loss : 0.5420, accuracy : 80.32
 3 Test Loss : 0.4999, accuracy : 81.77
 4 Test Loss : 0.4653, accuracy : 82.97
 5 Test Loss : 0.4448, accuracy : 83.66
 6 Test Loss : 0.4307, accuracy : 84.31
 7 Test Loss : 0.4218, accuracy : 84.60
 8 Test Loss : 0.4107, accuracy : 85.15
 9 Test Loss : 0.4022, accuracy : 85.34
10 Test Loss : 0.3993, accuracy : 85.39
11 Test Loss : 0.3881, accuracy : 85.97
12 Test Loss : 0.3861, accuracy : 86.07
13 Test Loss : 0.3783, accuracy : 86.29
14 Test Loss : 0.3732, accuracy : 86.49
15 Test Loss : 0.3735, accuracy : 86.40
16 Test Loss : 0.3725, accuracy : 86.60
17 Test Loss : 0.3639, accuracy : 86.82
18 Test Loss : 0.3610, accuracy : 86.88
19 Test Loss : 0.3598, accuracy : 86.78
20 Test Loss : 0.3570, accuracy : 87.27
21 Test Loss : 0.3534, accuracy : 87.16
22 Test Loss : 0.3486, accuracy : 87.28
23 Test Loss : 0.3470, accuracy : 87.53
24 Test Loss : 0.3481, accuracy : 87.32
25 Test Loss : 0.3423, accuracy : 87.69
